In [8]:
import simpy
import numpy as np
import random
import pandas as pd
import networkx as nx

# Constants and Simulation Parameters
KB_TO_MB = 1 / 1024
BIT_TO_MBIT = 1 / 1024
MHZ_TO_HZ = 1e6
ENV_RUN_TIME = 1000  # simulation run time
LC = 1e-24  # power usage constant
Y_NT = 0.5  # Weighted factor for local time cost
Y_NE = 0.5  # Weighted factor for energy cost

# Networking and Computational Parameters
transmission_rate = [500, 1000, 1500, 2000, 2500, 3000]  # in kb/s
num_devices_list = [50, 100, 150, 200, 250, 300]
task_data_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # task data in Mbit
max_power_consumption = 1000  # in Joules

# CPU and power parameters for devices and MEC server
local_device_cycle_freq = random.uniform(2, 4) * MHZ_TO_HZ  # in Hz
mec_server_cycle_freq = random.uniform(4, 8) * MHZ_TO_HZ  # in Hz
local_device_power = random.uniform(100, 400) / 1000  # in Watts
clock_cycles_per_bit = random.randint(800, 1200)

class MECServer:
    def __init__(self, env, capacity):
        self.env = env
        self.capacity = capacity
        self.processor = simpy.Resource(env, capacity=int(capacity))
        self.dag = nx.DiGraph()

    def add_task_to_dag(self, task_id, dependencies, task_data):
        self.dag.add_node(task_id, data=task_data, vk=random.random())
        for dep in dependencies:
            self.dag.add_edge(dep, task_id)

    def process_task(self, task_id):
        task_info = self.dag.nodes[task_id]
        data_bits = task_info['data'] * BIT_TO_MBIT * 1e6
        clock_cycles = data_bits * clock_cycles_per_bit

        local_time = clock_cycles / local_device_cycle_freq
        edge_time = clock_cycles / self.capacity
        total_task_time = (1 - task_info['vk']) * local_time + task_info['vk'] * edge_time
        
        yield self.env.timeout(total_task_time)
        return total_task_time

    def execute_tasks(self):
        for task in nx.topological_sort(self.dag):
            yield self.env.process(self.process_task(task))
            ct = self.env.now
            self.dag.nodes[task]['ct'] = ct

class Device:
    def __init__(self, env, mec_servers, logs, num_devices):
        self.env = env
        self.mec_servers = mec_servers
        self.logs = logs
        self.num_devices = num_devices

    def send_task(self, task_data, dependencies=[]):
        mec = min(self.mec_servers, key=lambda x: len(x.dag))
        task_id = self.env.now
        mec.add_task_to_dag(task_id, dependencies, task_data)

        # Calculate the local computation parameters
        Hn = task_data * BIT_TO_MBIT * 1e6  # Convert Mbit to bits
        Rn = clock_cycles_per_bit  # CPU cycles per bit
        Qn = random.uniform(0, 1)  # Offloading decision
        local_cycles = Hn * Rn * (1 - Qn)
        RTl = local_cycles / local_device_cycle_freq
        ATl = LC * local_device_cycle_freq**2 * local_cycles
        DTl = Y_NT * RTl + Y_NE * ATl

        selected_rate = random.choice(transmission_rate) * 1000  # Select a rate and convert kb/s to b/s
        transmission_time = (Hn * Qn) / selected_rate
        energy_consumed = transmission_time * local_device_power + ATl

        yield self.env.timeout(transmission_time + RTl)
        yield self.env.process(mec.execute_tasks())

        ct = mec.dag.nodes[task_id]['ct']
        total_energy = energy_consumed + (ct - self.env.now) * local_device_power

        self.logs.append({
            'num_devices': self.num_devices,
            'task_data': task_data,
            'completion_time': ct,
            'energy_consumed': total_energy,
            'weighted_waste': DTl,
            'transmission_rate': selected_rate 
        })

env = simpy.Environment()
mecs = [MECServer(env, mec_server_cycle_freq) for _ in num_devices_list]
logs = []

for num_devices in num_devices_list:
    devices = [Device(env, mecs, logs, num_devices) for _ in range(num_devices)]
    for device in devices:
        task_size = random.choice(task_data_sizes)
        env.process(device.send_task(task_size))

env.run(until=ENV_RUN_TIME)

df_logs = pd.DataFrame(logs)
print(df_logs.head())
if not df_logs.empty and df_logs['energy_consumed'].max() > max_power_consumption:
    print("Warning: Power consumption exceeded the maximum limit.")


   num_devices  task_data  completion_time  energy_consumed  weighted_waste  \
0           50         10         3.943298         0.001141        0.557581   
1          100         15         5.765738         0.001560        0.006335   
2          100         35         5.769678         0.004558        0.002686   
3          250         20         5.772930         0.002079        0.008958   
4          250         10         5.777651         0.000862        0.013601   

   transmission_rate  
0            1500000  
1            2500000  
2            2000000  
3            2500000  
4            3000000  


In [9]:
# Compute the metrics
mean_completion_by_devices = df_logs.groupby('num_devices')['completion_time'].mean()
mean_completion_by_task_data = df_logs.groupby('task_data')['completion_time'].mean()
mean_completion_by_transmission_rate = df_logs.groupby('transmission_rate')['completion_time'].mean()

energy_by_devices = df_logs.groupby('num_devices')['energy_consumed'].sum()
energy_by_task_data = df_logs.groupby('task_data')['energy_consumed'].sum()
energy_by_transmission_rate = df_logs.groupby('transmission_rate')['energy_consumed'].sum()
# Output the results
print("Mean Completion Times by Number of Devices:")
print(mean_completion_by_devices)
print("\nMean Completion Times by Task Data:")
print(mean_completion_by_task_data)
print("\nMean Completion Times by Transmission Rate:")
print(mean_completion_by_transmission_rate)
print("\nTotal Energy Consumption by Number of Devices:")
print(energy_by_devices)
print("\nTotal Energy Consumption by Task Data:")
print(energy_by_task_data)
print("\nTotal Energy Consumption by Transmission Rate:")
print(energy_by_transmission_rate)

Mean Completion Times by Number of Devices:
num_devices
50     10.158549
100     9.396520
150     9.023845
200     9.697473
250     9.892588
300     9.556542
Name: completion_time, dtype: float64

Mean Completion Times by Task Data:
task_data
5      6.477274
10     7.281153
15     7.879775
20     8.859725
25     9.535630
30    10.181979
35    11.260894
40    11.989344
45    12.192238
Name: completion_time, dtype: float64

Mean Completion Times by Transmission Rate:
transmission_rate
500000     9.839407
1000000    9.749603
1500000    9.483649
2000000    9.915323
2500000    9.208502
3000000    9.532877
Name: completion_time, dtype: float64

Total Energy Consumption by Number of Devices:
num_devices
50     0.143314
100    0.241084
150    0.438899
200    0.560940
250    0.649760
300    0.861505
Name: energy_consumed, dtype: float64

Total Energy Consumption by Task Data:
task_data
5     0.062661
10    0.107334
15    0.225010
20    0.282695
25    0.248207
30    0.333589
35    0.431933
40   

In [10]:
import simpy
import numpy as np
import random
import pandas as pd
import networkx as nx

# Constants and Simulation Parameters
KB_TO_MB = 1 / 1024
BIT_TO_MBIT = 1 / 1024
MHZ_TO_HZ = 1e6
ENV_RUN_TIME = 1000  # simulation run time

# Networking and Computational Parameters
transmission_bandwidth = 20 * MHZ_TO_HZ  # 20 MHz
num_devices_list = [50, 100, 150, 200, 250, 300]
task_data_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # task data in Mbit
max_power_consumption = 1000  # in Joules

# CPU and power parameters for devices and MEC server
local_device_cycle_freq = random.uniform(2, 4) * MHZ_TO_HZ  # in Hz
mec_server_cycle_freq = random.uniform(4, 8) * MHZ_TO_HZ  # in Hz
local_device_power = random.uniform(100, 400) / 1000  # in Watts
clock_cycles_per_bit = random.randint(800, 1200)

class MECServer:
    def __init__(self, env, capacity):
        self.env = env
        self.capacity = capacity  # Convert GHz to Hz for calculations
        self.processor = simpy.Resource(env, capacity=int(capacity))
        self.dag = nx.DiGraph()

    def calculate_transmission_rate(self, RQx, dVx, P):
        B = 20 * 1e6  # 20 MHz bandwidth
        H0 = 1e-3  # noise spectral density
        return B * np.log2(1 + (RQx * dVx) / (H0 * P))

    def add_task_to_dag(self, task_id, dependencies, task_data):
        self.dag.add_node(task_id, data=task_data, vk=random.random())
        for dep in dependencies:
            self.dag.add_edge(dep, task_id)

    def process_task(self, task_id):
        task_info = self.dag.nodes[task_id]
        data_bits = task_info['data'] * BIT_TO_MBIT * 1e6
        clock_cycles = data_bits * clock_cycles_per_bit

        local_time = clock_cycles / local_device_cycle_freq
        edge_time = clock_cycles / self.capacity
        total_task_time = (1 - task_info['vk']) * local_time + task_info['vk'] * edge_time
        
        yield self.env.timeout(total_task_time)
        return total_task_time

    def execute_tasks(self):
        for task in nx.topological_sort(self.dag):
            yield self.env.process(self.process_task(task))
            ct = self.env.now  # Completion time of the current task
            self.dag.nodes[task]['ct'] = ct

class Device:
    def __init__(self, env, mec_servers, logs, num_devices):
        self.env = env
        self.mec_servers = mec_servers
        self.logs = logs
        self.num_devices = num_devices

    def send_task(self, task_data, dependencies=[]):
        mec = min(self.mec_servers, key=lambda x: len(x.dag))
        task_id = self.env.now  # Use current simulation time as unique task ID
        mec.add_task_to_dag(task_id, dependencies, task_data)

        data_bits = task_data * BIT_TO_MBIT * 1e6
        transmission_time = data_bits / transmission_bandwidth
        energy_consumed = transmission_time * local_device_power

        yield self.env.timeout(transmission_time)
        yield self.env.process(mec.execute_tasks())

        ct = mec.dag.nodes[task_id]['ct']
        total_energy = energy_consumed + (ct - self.env.now) * local_device_power

        self.logs.append({
            'num_devices': self.num_devices,
            'task_data': task_data,
            'completion_time': ct,
            'energy_consumed': total_energy
        })

# Simulation Environment Setup
env = simpy.Environment()
mecs = [MECServer(env, mec_server_cycle_freq) for _ in num_devices_list]
logs = []

# Instantiate devices and start simulation
for num_devices in num_devices_list:
    devices = [Device(env, mecs, logs, num_devices) for _ in range(num_devices)]
    for device in devices:
        task_size = random.choice(task_data_sizes)
        env.process(device.send_task(task_size))

env.run(until=ENV_RUN_TIME)

# Post-Simulation Analysis
df_logs = pd.DataFrame(logs)
print(df_logs.head())
if not df_logs.empty and df_logs['energy_consumed'].max() > max_power_consumption:
    print("Warning: Power consumption exceeded the maximum limit.")


   num_devices  task_data  completion_time  energy_consumed
0           50          5         1.016015         0.000067
1           50          5         1.083142         0.000067
2           50         10         2.648634         0.000133
3           50         20         4.678961         0.000267
4           50          5         9.259962         0.000067


In [12]:
# Compute the metrics
mean_completion_by_devices = df_logs.groupby('num_devices')['completion_time'].mean()
mean_completion_by_task_data = df_logs.groupby('task_data')['completion_time'].mean()
#mean_completion_by_transmission_rate = df_logs.groupby('transmission_rate')['completion_time'].mean()

energy_by_devices = df_logs.groupby('num_devices')['energy_consumed'].sum()
energy_by_task_data = df_logs.groupby('task_data')['energy_consumed'].sum()
#energy_by_transmission_rate = df_logs.groupby('transmission_rate')['energy_consumed'].sum()
# Output the results
print("Mean Completion Times by Number of Devices:")
print(mean_completion_by_devices)
print("\nMean Completion Times by Task Data:")
print(mean_completion_by_task_data)
print("\nMean Completion Times by Transmission Rate:")
#print(mean_completion_by_transmission_rate)
print("\nTotal Energy Consumption by Number of Devices:")
print(energy_by_devices)
print("\nTotal Energy Consumption by Task Data:")
print(energy_by_task_data)
print("\nTotal Energy Consumption by Transmission Rate:")
#print(energy_by_transmission_rate)

Mean Completion Times by Number of Devices:
num_devices
50     8.761994
100    9.260927
150    9.260939
200    9.260902
250    9.260850
300    9.260948
Name: completion_time, dtype: float64

Mean Completion Times by Task Data:
task_data
5     9.134613
10    9.200101
15    9.260450
20    9.225450
25    9.260939
30    9.261183
35    9.261427
40    9.261671
45    9.285912
Name: completion_time, dtype: float64

Mean Completion Times by Transmission Rate:

Total Energy Consumption by Number of Devices:
num_devices
50     0.017539
100    0.033011
150    0.050017
200    0.064689
250    0.077293
300    0.100767
Name: energy_consumed, dtype: float64

Total Energy Consumption by Task Data:
task_data
5     0.008736
10    0.014672
15    0.023008
20    0.034678
25    0.039347
30    0.044415
35    0.053685
40    0.058153
45    0.066623
Name: energy_consumed, dtype: float64

Total Energy Consumption by Transmission Rate:
